<a href="https://colab.research.google.com/github/Charmlessmann/colab/blob/main/detection_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U --pre tensorflow=="2.2"
!pip install tf_slim

!pip install pycocotools


Requirement already up-to-date: tensorflow==2.2 in /usr/local/lib/python3.6/dist-packages (2.2.0)
  Using cached https://files.pythonhosted.org/packages/1d/74/0a6fcb206dcc72a6da9a62dd81784bfdbff5fedb099982861dc2219014fb/tensorboard-2.2.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a4/f5/926ae53d6a226ec0fda5208e0e581cffed895ccc89e36ba76a8e60895b78/tensorflow_estimator-2.2.0-py2.py3-none-any.whl
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0


In [2]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [3]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [4]:
%%bash 
cd models/research
pip install .

Processing /content/models/research
  Using cached https://files.pythonhosted.org/packages/c2/14/3724e7316a065b79174b10e34471e21ebb6896efbb82f689651584e6f5da/tensorflow-2.4.1-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/74/7e/622d9849abf3afb81e482ffc170758742e392ee129ce1540611199a59237/tensorflow_estimator-2.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/64/21/eebd23060763fedeefb78bc2b286e00fa1d8abda6f70efa2ee08c28af0d4/tensorboard-2.4.1-py3-none-any.whl
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1617397 sha256=39ce5cffcd22e271ecac8d7c69a191881e9e2615beddeb04015d6bcbc2e57fd4
  Stored in directory: /tmp/pip-ephem-wheel-cache-0bs5sypt/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-

In [5]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from six import BytesIO

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model


# List of the strings that is used to add correct label for each box.

PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/workspace/test_image')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

print(detection_model.signatures['serving_default'].inputs)

detection_model.signatures['serving_default'].output_dtypes

detection_model.signatures['serving_default'].output_shapes

def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    output_dict['detection_masks'] = tf.convert_to_tensor(output_dict['detection_masks'], dtype=tf.float32)
    output_dict['detection_boxes'] = tf.convert_to_tensor(output_dict['detection_boxes'], dtype=tf.float32)
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
  return output_dict


    



76537856/76534733 [==============================] - 1s 0us/step
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


In [6]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


In [10]:
import os

def visitDir(path):
    if not os.path.isdir(path):
        print('Error: "', path, '" is not a directory or does not exist.')
        return
    else:
        global x
        try:
            for lists in os.listdir(path):
                sub_path = os.path.join(path, lists)
                x += 1
                print('No.', x, ' ', sub_path)
                if os.path.isdir(sub_path):
                    visitDir(sub_path)
        except:
            pass


if __name__ == '__main__':
    x = 0
    visitDir('patients/18L')
    print('Total Permission Files: ', x)

No. 1   patients/18L/hand-00214.jpg
No. 2   patients/18L/hand-00157.jpg
No. 3   patients/18L/hand-00300.jpg
No. 4   patients/18L/hand-00301.jpg
No. 5   patients/18L/hand-00485.jpg
No. 6   patients/18L/hand-00273.jpg
No. 7   patients/18L/hand-00379.jpg
No. 8   patients/18L/hand-00463.jpg
No. 9   patients/18L/hand-00187.jpg
No. 10   patients/18L/hand-00295.jpg
No. 11   patients/18L/hand-00594.jpg
No. 12   patients/18L/hand-00518.jpg
No. 13   patients/18L/hand-00363.jpg
No. 14   patients/18L/hand-00486.jpg
No. 15   patients/18L/hand-00437.jpg
No. 16   patients/18L/hand-00327.jpg
No. 17   patients/18L/hand-00111.jpg
No. 18   patients/18L/hand-00562.jpg
No. 19   patients/18L/hand-00284.jpg
No. 20   patients/18L/hand-00401.jpg
No. 21   patients/18L/hand-00058.jpg
No. 22   patients/18L/hand-00220.jpg
No. 23   patients/18L/hand-00337.jpg
No. 24   patients/18L/hand-00051.jpg
No. 25   patients/18L/hand-00185.jpg
No. 26   patients/18L/hand-00395.jpg
No. 27   patients/18L/hand-00521.jpg
No. 28   p

In [ ]:
import scipy.io as sio

test_image_dir = 'patients/18L'
test_images_np = []
for i in range(1,x+1):
    path = os.path.join(test_image_dir, ('hand-%05d' % i) + ".jpg")
    test_images_np.append(np.expand_dims(
        load_image_into_numpy_array(path), axis=0))


In [ ]:

box_arr = []


for i in range(len(test_images_np)):
  output_dict = run_inference_for_single_image(detection_model, test_images_np[i][0])
  
  boxes = np.squeeze(output_dict['detection_boxes'])
  scores = np.squeeze(output_dict['detection_scores'])
  
  #set a min thresh score, say 0.8
  min_score_thresh = np.amax(scores)-0.09
  bboxes = boxes[scores > min_score_thresh]
  #get image size
  im_width, im_height = 1920,1080
  final_box = []
  for box in bboxes:
    ymin, xmin, ymax, xmax = box
    final_box.append([xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height])

  box_arr = np.append(box_arr, final_box)
print(box_arr)
# sio.savemat(r'models/research/object_detection/workspace/test_video/result/box.mat', mdict={'arr': box_arr})
sio.savemat('box/P18_L.mat', mdict={'arr': box_arr})
